In [3]:
import pandas as pd
from helpers import read_csv, load_pickled_data, pickle_data
import os
import matplotlib.pyplot as plt 
import plotly
from helpers import create_nx_graph
import plotly.graph_objects as go
import os, sys

from helpers import bfs, get_class_pairs, choose_node_based_on_neighbors, load_pickled_data
import numpy as np
import networkx as nx

importances_dict:dict = load_pickled_data("feature_importances.pkl")
feat_occurences:dict = load_pickled_data('frequencies.pkl')
print(feat_occurences)
importances = {}
names = []
i = 0
for key, value in importances_dict.items():
    importances[i] = value
    names.append(key)


keep_cols = ["txId", 'Time step'] + names
TIMESTEPS =[13, 42, 35, 32, 29, 22, 9 ,20, 16, 15, 17, 20, 21, 24, 25, 26, 29, 31, 38, 40, 41, 42]

DATASET_BASE = "dataset"
PICKLED_TIMESTEPS = os.path.join("exports", "split_on_timestep.pkl")
txs_features:pd.DataFrame =  read_csv(os.path.join(DATASET_BASE, "elliptic_txs_features.csv"), ret_Dataframe=True)[keep_cols]



{'Local_feature_2': {46: 1, 45: 1, 15: 330, 25: 238, 44: 18, 28: 86, 34: 189, 21: 37, 30: 925, 38: 12, 37: 572, 14: 736, 36: 119, 8: 70, 23: 214, 32: 26, 29: 112, 13: 19, 39: 2, 42: 4, 18: 329, 6: 13, 43: 41, 5: 175, 26: 54, 33: 20, 9: 31, 31: 18, 4: 5, 3: 18, 24: 7, 41: 6, 35: 3, 16: 22, 7: 18, 27: 7, 20: 3, 11: 8, 10: 11, 40: 1, 12: 6, 1: 7, 2: 1, 22: 1}, 'Local_feature_5': {13: 14, 1: 3315, 2: 293, 0: 677, 4: 145, 10: 20, 9: 16, 6: 8, 12: 3, 5: 13, 8: 6, 7: 6}, 'Local_feature_18': {8: 82, 5: 27, 14: 212, 19: 708, 9: 89, 1: 694, 3: 42, 28: 830, 23: 385, 38: 86, 40: 101, 21: 338, 34: 22, 36: 19, 26: 72, 50: 35, 13: 64, 45: 11, 22: 154, 2: 1, 20: 138, 6: 12, 49: 8, 32: 9, 41: 24, 25: 59, 52: 20, 31: 10, 12: 32, 35: 47, 39: 15, 42: 12, 15: 25, 51: 15, 10: 31, 4: 15, 46: 24, 37: 3, 7: 6, 47: 8, 48: 1, 11: 4, 18: 1, 33: 2, 53: 4, 44: 8, 54: 1, 17: 1, 24: 1, 29: 1, 30: 3, 43: 2, 16: 1, 27: 1}, 'Local_feature_41': {0: 2480, 21: 453, 17: 277, 16: 97, 19: 151, 5: 77, 9: 36, 28: 315, 33: 82, 1

In [4]:
txs_features = txs_features[keep_cols]
txs_classes:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_classes.csv"), ret_Dataframe=True)
txs_edgelist:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_edgelist.csv"), ret_Dataframe=True)


merge_feats_class = pd.merge(txs_features, txs_classes, on='txId').dropna()
merge_feats_class.shape
print(f"Total Nodes df shape: {txs_features.shape}")
print(f"total edges df shape: {txs_edgelist.shape}")
#timestep_graphs = load_pickled_data(PICKLED_TIMESTEPS)
timestep_dataframe = merge_feats_class.loc[merge_feats_class['Time step'].isin(TIMESTEPS)].reset_index(drop=True)
timestep_edges = txs_edgelist[txs_edgelist['txId1'].isin(timestep_dataframe['txId'])].reset_index(drop=True)

from helpers import discretize_per_column
freq, timestep_dataframe = discretize_per_column(timestep_dataframe, 7, 0, True)
timestep_cols = timestep_dataframe.columns
timestep_list = timestep_dataframe.values.tolist()

edge_list = timestep_edges.values.tolist()

from Graph import Graph, Node
from visualizations import visualize_graph_from_list, visualize_graph_df

graph_of_interest = Graph()
graph_of_interest.create_from_lists(timestep_cols, timestep_list, edge_list)
graph_of_interest.feature_importances =  importances_dict
graph_of_interest.feature_occurences = freq
pickle_data("working_graph2.pkl", graph_of_interest, True)

# g:nx.DiGraph = create_nx_graph(timestep_dataframe, timestep_edges)

Total Nodes df shape: (203769, 31)
total edges df shape: (234355, 2)
Local_feature_2
{232: 1915, 164: 4642, 115: 1135, 208: 369, 30: 4620, 210: 606, 143: 485, 106: 301, 29: 582, 93: 4813, 132: 172, 14: 1159, 35: 31, 70: 495, 85: 369, 114: 1992, 7: 40, 170: 2021, 166: 234, 116: 376, 121: 102, 96: 8, 137: 167, 125: 460, 4: 118, 26: 130, 109: 314, 180: 317, 101: 973, 189: 736, 5: 317, 105: 223, 126: 128, 165: 917, 10: 6, 88: 1212, 84: 685, 90: 271, 112: 146, 92: 127, 45: 211, 82: 156, 151: 62, 46: 450, 33: 189, 155: 194, 8: 31, 159: 1730, 91: 1013, 20: 23, 153: 181, 160: 14, 147: 3581, 31: 111, 134: 467, 186: 832, 11: 210, 233: 2294, 145: 20, 188: 697, 73: 1219, 27: 55, 1: 152, 52: 16, 103: 43, 16: 141, 74: 447, 72: 187, 47: 216, 13: 39, 18: 65, 69: 13, 110: 410, 54: 111, 237: 338, 150: 104, 182: 324, 32: 38, 86: 573, 71: 77, 6: 232, 218: 1258, 211: 797, 127: 15, 25: 76, 23: 27, 99: 170, 12: 30, 117: 359, 223: 128, 119: 634, 24: 20, 171: 401, 108: 15, 212: 547, 51: 121, 194: 931, 9: 101, 

In [28]:
graph_dict = {}
for timestep in set(timestep_dataframe["Time step"].values):
    single_timestep_dataframe = timestep_dataframe.loc[timestep_dataframe['Time step'].isin([timestep])].reset_index(drop=True)
    single_timestep_edges = timestep_edges[timestep_edges['txId1'].isin(single_timestep_dataframe['txId'])].reset_index(drop=True)
    t_cols = single_timestep_dataframe.columns
    t_list = single_timestep_dataframe.values.tolist()
    e_list = single_timestep_edges.values.tolist()
    print(len(t_list))
    g = Graph()
    g.create_from_lists(t_cols, t_list, e_list)
    g.feature_importances =  importances_dict
    g.feature_occurences = freq
    graph_dict[timestep] = g
    
pickle_data("working_graphs_dict.pkl", graph_dict, True)


4991
4516
3623
2963
3373
4287
3534
5878
4580
2307
2519
4266
2775
4513
5486
2874
4458
5312
7125
Succesfully pickled the object on file working_graphs_dict.pkl


In [26]:
print(set(freq["Local_feature_2"]))


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 2